<a href="https://colab.research.google.com/github/good0neighbour/ESTCampAI/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/ex02_%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EA%B8%B0%EB%8A%A5_%EA%B5%AC%ED%98%84_Collaborative_Filtering(%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ESTCampAI/추천시스템

/content/drive/MyDrive/ESTCampAI/추천시스템


### 콘텐츠 기반 필터링의 한계와 협업 필터링
- 콘텐츠 기반 필터링의 한계점 극복
  - 콘텐츠 기반 필터링은 아이템 내재적 정보에 의존하므로, 사용자의 잠재적인 취향이나 다른 사용자와의 상호작용 정보를 반영하기는 어려움
- 협업 필터링:
  - 사용자 행동 기반 : 여러 사용자의 평가나 소비 데이터를 활용하여 사용자 간 혹은 아이템 간 유사도를 산출
  - 다양한 추천 : 콘텐츠 기반의 한계를 보완하여 사용자의 다양한 취향과 새로운 아이템을 추천 가능
  - 하이브리드 접근 : 콘텐츠 기반과 협업 필터링을 결합하여 더욱 정교한 추천 시스템을 구현할 수 있다(ex. Youtube)

- 취향이 비슷한 친구들에게 물어보는 것과 유사한 방식
- 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식을 기반으로 추천을 수행하는 것

> 일반적으로 사용자 기반보다는 아이템 기반 정확도 더 높음
>- 비슷한 영화(또는 상품)을 좋아한다고 해서 사람의 취향이 비슷하다고 판단하기 어려운 경우 많음

In [3]:
import numpy as np
import pandas as pd
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

- 데이터셋 출처 : https://grouplens.org/datasets/movielens/latest/
- Grouplens 사이트에서 만든 MovieLens 데이터셋 활용

In [4]:
movies.info()
display(movies.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


- movieID : 영화 고유 ID, 각 영화의 식별자, 다른 데이터셋과 연결할 때 사용(PK)
- title : 영화 제목(출시년도)
- genres : 영화 장르 정보, | 기호로 구분되어 기록됨

In [5]:
ratings.info()
display(ratings.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


- userId : 사용자의 고유 ID, 평점을 남긴 사용자 식별을 위해 사용
- movieId : 영화 고유 ID, movies 데이터셋의 movieId와 연결되어 영화 정보 참조에 활용(FK)
- rating : 사용자가 영화에 부여한 평점
- timestamp : 평점이 기록된 시각

In [6]:
# 추천에 필요없는 timestamp(평점이 기록된 시각) 제외
ratings = ratings.drop(columns = 'timestamp')
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


### 1. 사용자-아이템 평점 행렬 생성
- 1.1 사용자-아이템 평점 행렬 생성
- 1.2 결측치 채우

### 사용자-아이템 행렬:
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbtgDyd%2FbtryDKJCNjV%2FlMQwtLJBGuVEJhN28up0nK%2Fimg.png"/>

- 협업 필터링 기반은 최근접 이웃 방식과 잠재 요인 방식으로 나뉘는데, 두 방식 모두 사용자-아이템 행렬에 의지해서 추천을 수행

> **사용자-아이템 평점 행렬**
>- 각 행에는 사용자를, 각 열 아이템(영화, 상품 등)을 나타내며, 셀에는 해당 사용자가 해당 아이템에 대해 매긴 평점이나 구매, 클릭 등 상호작용 데이터가 기록된 행렬
>- NaN은 해당 아이템에 대한 평가가 없는 상태를 의미

In [7]:
ratings.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [8]:
# 영화 제목 정보를 추가하기 위해 movies 데이터셋과 ratings 데이터셋을 movieId를 기준으로 병합
rating_movies = pd.merge(ratings, movies, on = 'movieId')
rating_movies.head(3)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller


In [10]:
ratings_matrix = rating_movies.pivot_table(
    index = 'userId', # 행 : 사용자 아이디
    columns = 'title', # 열 : 영화 제목
    values = 'rating' # rkqt : 평점
)

display(ratings_matrix.head(3))
ratings_matrix.shape
# 행 : 610명의 고유 사용자 아이디
# 열 : 9719개의 고유 영화 제목
# ex) 1번 사용자가 "i Three Amigos! (1986)"를 보고 4.0을 부여함 (평점을 매기지 않은 경우 NaN)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(610, 9719)

- 1.2 결측치 채우기

In [11]:
# NaN 값은 해당 사용자가 해당 영화를 평가하지 않은 것이므로, 0으로 채워줌
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
